In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [2]:
def create_link(ecli_from, ecli_to, sparql):
    base_query = '''
    prefix dcterm: <http://purl.org/dc/terms/>
    insert data 
    {{<http://deeplink.rechtspraak.nl/uitspraak?id={}>
        dcterm:references 
        <http://deeplink.rechtspraak.nl/uitspraak?id={}> 
    .}}'''
    query = base_query.format(ecli_from, ecli_to)
    sparql.method = 'POST'
    sparql.setQuery(query)
    res = sparql.query()

In [3]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [6]:
#Create for Gijs' network

import pandas as pd
filepath_csv = '/media/sf_VBox_Shared/CaseLaw/graphs/test3 wg aanspr.csv'
links_wgaans = pd.read_csv(filepath_csv, sep=';', header=0)
links_wgaans.columns = ['id', 'reference']
links_wgaans = links_wgaans.drop_duplicates()

In [33]:
for i, r in links_wgaans.iterrows():
    create_link(r['id'], r['reference'], sparql)

In [4]:
# Create for previously found links
import sqlite3

conn = sqlite3.connect('rechtspraak.db')
c = conn.cursor()

In [7]:
links = pd.DataFrame(c.execute('''select l.id, l.reference
            from uitspraken_links_ljn l join 
                ( select id from uitspraken_meta ) s
                on s.id = l.reference
        ''').fetchall(), columns=['id', 'reference'])

In [8]:
conn.close()

In [9]:
for i, r in links.iterrows():
    create_link(r['id'], r['reference'], sparql)

In [10]:
#Make a query
queryString = "prefix dcterm: <http://purl.org/dc/terms/> select (COUNT(?s) as ?count) {?s dcterm:references ?o}"
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
ret = sparql.query()
ret.convert()

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'type': 'literal',
     'value': '5939'}}]}}

In [12]:
print(links_wgaans.shape)
print(links.shape)

(177, 2)
(2502, 2)


In [16]:
links.head()

,id,reference
0,ECLI:NL:HR:2013:1036,ECLI:NL:HR:2007:BA1522
1,ECLI:NL:HR:2013:1040,ECLI:NL:HR:2011:BP2314
2,ECLI:NL:HR:2013:1042,ECLI:NL:HR:2013:BZ3640
3,ECLI:NL:HR:2013:1047,ECLI:NL:HR:2012:BV0229
4,ECLI:NL:HR:2013:1048,ECLI:NL:HR:2012:BV0258
